In [4]:
import torch
from torchtext.data import Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe

# FILEPATH: sentiment_classifier.py

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the model architecture for Vanilla RNNs
class VanillaRNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(VanillaRNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        hidden = torch.squeeze(hidden)
        return self.fc(hidden)

# Define the model architecture for LSTMs
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, _) = self.lstm(embedded)
        hidden = torch.squeeze(hidden)
        return self.fc(hidden)

# Load the IMDb dataset
TEXT = Field(tokenize='spacy', lower=True)
LABEL = Field(sequential=False, is_target=True)

train_data, test_data = TabularDataset.splits(
    path='Assignment 2\IMDB Dataset.csv',
    train='train.csv',
    test='test.csv',
    format='csv',
    fields=[('text', TEXT), ('label', LABEL)]
)

# Build the vocabulary using GloVe embeddings
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
input_dim = len(TEXT.vocab)
embedding_dim = 300
hidden_dim = 256
output_dim = 1
batch_size = 64

# Create iterators for the train and test datasets
train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size=batch_size,
    device=device
)

# Initialize the models
vanilla_rnn_model = VanillaRNN(input_dim, embedding_dim, hidden_dim, output_dim).to(device)
lstm_model = LSTM(input_dim, embedding_dim, hidden_dim, output_dim).to(device)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer_vanilla_rnn = optim.Adam(vanilla_rnn_model.parameters())
optimizer_lstm = optim.Adam(lstm_model.parameters())

# Train the Vanilla RNN model
def train_vanilla_rnn(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, batch.label.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Train the LSTM model
def train_lstm(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, batch.label.float())
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Train the Vanilla RNN model
for epoch in range(num_epochs):
    train_loss = train_vanilla_rnn(vanilla_rnn_model, train_iterator, optimizer_vanilla_rnn, criterion)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}')

# Train the LSTM model
for epoch in range(num_epochs):
    train_loss = train_lstm(lstm_model, train_iterator, optimizer_lstm, criterion)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}')

# Repeat [1] and [2] with on-the-fly embeddings using torch
class OnTheFlyEmbeddingRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(OnTheFlyEmbeddingRNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = nn.Embedding.from_pretrained(text)
        output, hidden = self.rnn(embedded)
        hidden = torch.squeeze(hidden)
        return self.fc(hidden)

class OnTheFlyEmbeddingLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(OnTheFlyEmbeddingLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = nn.Embedding.from_pretrained(text)
        output, (hidden, _) = self.lstm(embedded)
        hidden = torch.squeeze(hidden)
        return self.fc(hidden)

# Initialize the models with on-the-fly embeddings
on_the_fly_embedding_rnn_model = OnTheFlyEmbeddingRNN(input_dim, hidden_dim, output_dim).to(device)
on_the_fly_embedding_lstm_model = OnTheFlyEmbeddingLSTM(input_dim, hidden_dim, output_dim).to(device)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer_on_the_fly_embedding_rnn = optim.Adam(on_the_fly_embedding_rnn_model.parameters())
optimizer_on_the_fly_embedding_lstm = optim.Adam(on_the_fly_embedding_lstm_model.parameters())

# Train the on-the-fly embedding Vanilla RNN model
for epoch in range(num_epochs):
    train_loss = train_vanilla_rnn(on_the_fly_embedding_rnn_model, train_iterator, optimizer_on_the_fly_embedding_rnn, criterion)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}')

# Train the on-the-fly embedding LSTM model
for epoch in range(num_epochs):
    train_loss = train_lstm(on_the_fly_embedding_lstm_model, train_iterator, optimizer_on_the_fly_embedding_lstm, criterion)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}')

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Program Files\Python311\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [5]:
import torch